In [1]:
print("asdf")

asdf


In [ ]:
%load_ext autoreload
%autoreload 2

from datetime import datetime
from pathlib import Path

import matplotlib.pyplot as plt

from coralshift.utils import file_ops
from coralshift.dataloading import config, get_data

import warnings
warnings.filterwarnings(action="ignore", category=RuntimeWarning)

# # gt_data dir
gt_data_dir = config.gt_data_dir

config_info = file_ops.read_yaml("/maps/rt582/coralshift/runs/config_files/config_test.yaml")

from dask.distributed import get_client, Client, LocalCluster

cluster = LocalCluster(n_workers=4)
client = Client(cluster)
# client.close()
# cluster.close()


from coralshift.plotting import spatial_plots
from coralshift import functions_creche

In [ ]:
curacao_config_info = file_ops.read_yaml("/maps/rt582/coralshift/runs/config_files/curacao.yaml") 

get_data.ProcessML(config_info=curacao_config_info).get_merged_datasets()

In [ ]:
config_info = file_ops.read_yaml("/maps/rt582/coralshift/runs/config_files/config_test.yaml")

(trains, tests, vals), ds_info = get_data.ProcessML(
    model_data=config_info["ds_type"],
    datasets=config_info["datasets"],
    train_test_val_frac=config_info["train_test_val_frac"],
    X_scaler=config_info["predictor_scaler"],
    y_scaler=config_info["predictand_scaler"],
    split_type=config_info["split_type"],
    depth_mask_lims=config_info["depth_mask_lims"],
    lats=config_info["lats"],
    lons=config_info["lons"],
    levs=config_info["levs"],
    resolution=config_info["resolution"],
    pos_neg_ratio=config_info["pos_neg_ratio"],
    resolution_unit=config_info["resolution_unit"],
    upsample_method=config_info["upsample_method"],
    downsample_method=config_info["downsample_method"],
    env_vars=config_info["env_vars"],
    predictand=config_info["predictand"],
    random_state=config_info["random_state"],
).generate_ml_ready_data()

In [ ]:
config_info = file_ops.read_yaml("/maps/rt582/coralshift/runs/config_files/config_test.yaml")

model = get_data.RunML(
    trains=trains,
    tests=tests,
    vals=vals,
    # regressor_classification_threshold=config_info["regressor_classification_threshold"],
    # n_samples=config_info["hyperparameter_search"]["n_samples"],
    # model_code=config_info["model_code"],
    # cv_folds=config_info["hyperparameter_search"]["cv_folds"],
    # n_iter=config_info["hyperparameter_search"]["n_iter"],
    # do_search=config_info["hyperparameter_search"]["do_search"],
    # do_save_model=config_info["do_save_model"],
    # do_train=config_info["do_train"],
    # param_search_type=config_info["hyperparameter_search"]["type"],
    config_info=config_info,
    additional_info=ds_info).run_model()

In [ ]:
y = tests[1]
predictions = get_data.AnalyseResults(
    model=model,
    trains=trains,
    tests=tests,
    vals=vals,
    config_info=file_ops.read_yaml("/maps/rt582/coralshift/runs/0-05d/xgb_reg/ID005_cmip6_unep_gebco_CONFIG.yaml")).make_predictions(tests[0])

conf_vals = get_data.AnalyseResults(
    model=model,
    trains=trains,
    tests=tests,
    vals=vals,
    config_info=file_ops.read_yaml("/maps/rt582/coralshift/runs/0-05d/xgb_reg/ID005_cmip6_unep_gebco_CONFIG.yaml")).produce_plots(y, predictions)

In [ ]:
get_data.AnalyseResults(
    model=model,
    trains=trains,
    tests=tests,
    vals=vals,
    config_info=file_ops.read_yaml("/maps/rt582/coralshift/runs/0-05d/xgb_reg/ID005_cmip6_unep_gebco_CONFIG.yaml")).analyse_results()

In [ ]:
from coralshift.processing import spatial_data
import cartopy.crs as ccrs

result = spatial_data.spatial_predictions_from_data(y, predictions)

fig, axes = plt.subplots(ncols=2, figsize=[20,10], subplot_kw={"projection": ccrs.PlateCarree()})
spatial_plots.plot_spatial(result["label"], fax=(fig,axes[0]))
spatial_plots.plot_spatial(result["predictions"], fax=(fig,axes[1]))

In [ ]:
max(y)

In [ ]:
predictions

In [ ]:
sum(sum(conf_vals.values == 4))

In [ ]:
plt.hist(conf_vals.values.flatten())

In [ ]:
get_data.AnalyseResults(
    model=model,
    trains=trains,
    tests=tests,
    vals=vals,
    config_info=file_ops.read_yaml("/maps/rt582/coralshift/runs/0-05d/xgb_reg/ID003_cmip6_unep_gebco_CONFIG.yaml")).analyse_results()

In [ ]:
file_ops.read_pkl("/maps/rt582/coralshift/runs/0-05d/xgb_reg/ID003_cmip6_unep_gebco_MODEL.pickle")

In [ ]:
model.get_score()

In [ ]:
trains([0]["elevation"])

In [ ]:
trains[0]["elevation"]

In [ ]:
merged = get_data.ProcessML(
    model_data=config_info["ds_type"],
    datasets=config_info["datasets"],
    train_test_val_frac=config_info["train_test_val_frac"],
    X_scaler=config_info["predictor_scaler"],
    y_scaler=config_info["predictand_scaler"],
    split_type=config_info["split_type"],
    depth_mask_lims=config_info["depth_mask_lims"],
    lats=config_info["lats"],
    lons=config_info["lons"],
    levs=config_info["levs"],
    resolution=config_info["resolution"],
    pos_neg_ratio=config_info["pos_neg_ratio"],
    resolution_unit=config_info["resolution_unit"],
    upsample_method=config_info["upsample_method"],
    downsample_method=config_info["downsample_method"],
    env_vars=config_info["env_vars"],
    predictand=config_info["predictand"],
    random_state=config_info["random_state"],
).get_merged_datasets()

In [ ]:
spatial_plots.plot_spatial(merged["mlotst_january_mean"])

In [ ]:
# process data for ml
config_info = file_ops.read_yaml("/maps/rt582/coralshift/config_test.yaml")

trains, tests, vals = get_data.ProcessML(
    model_data=config_info["ds_type"],
    datasets=config_info["datasets"],
    train_test_val_frac=config_info["train_test_val_frac"],
    X_scaler=config_info["predictor_scaler"],
    y_scaler=config_info["predictand_scaler"],
    split_type=config_info["split_type"],
    depth_mask_lims=config_info["depth_mask_lims"],
    lats=config_info["lats"],
    lons=config_info["lons"],
    levs=config_info["levs"],
    resolution=config_info["resolution"],
    pos_neg_ratio=config_info["pos_neg_ratio"],
    resolution_unit=config_info["resolution_unit"],
    upsample_method=config_info["upsample_method"],
    downsample_method=config_info["downsample_method"],
    env_vars=config_info["env_vars"],
    predictand=config_info["predictand"],
    random_state=config_info["random_state"],
).generate_ml_ready_data()

In [ ]:
trains[1]

In [ ]:
functions_creche.calc_non_zero_ratio(trains[1])

In [ ]:
out = functions_creche.pd_to_array(trains[1])
spatial_plots.plot_spatial(out, title="UNEP_GDCR")

In [ ]:
config_info = file_ops.read_yaml("/maps/rt582/coralshift/config_test.yaml")
# raster = get_data.ReturnRaster(dataset="cmip6", config_info=config_info).get_raw_raster()
resampled = get_data.ReturnRaster(dataset="gebco", config_info=config_info).return_raster()
# spatial_plots.plot_spatial(resampled["mlotst_january_mean"])

In [ ]:
cmip = get_data.ReturnRaster(
    dataset="cmip6",
    config_info=config_info).return_raster()
# cmip["mlotst_january_mean"].plot()

spatial_plots.plot_spatial(cmip["mlotst_january_mean"])

In [ ]:
cmip

In [ ]:
cmip = get_data.ReturnRaster(
    dataset="cmip6",
    config_info=config_info).get_raw_raster()
cmip = get_data.ReturnRaster(
    dataset="cmip6",
    config_info=config_info).get_spatially_buffered_raster(cmip)

spatial_plots.plot_spatial(cmip["mlotst_january_mean"], figsize=[2,2])

In [ ]:
cmip.isel(latitude=slice(1,6), longitude=slice(1,6))

In [ ]:
cmip["mlotst_april_max"].isel(latitude=slice(100,105), longitude=slice(100,105))

In [ ]:
spatial_plots.plot_spatial(cmip["mlotst_april_max"]
    .isel(latitude=slice(100,105), longitude=slice(100,105))
    )

In [ ]:
lats = cmip.latitude.values
lons = cmip.longitude.values

from coralshift.processing import spatial_data

regrid_cmip = functions_creche.xesmf_regrid(cmip, lat_range=(min(lats), max(lats)), lon_range=(min(lons), max(lons)), resolution=1)
# regrid_cmip = regrid_cmip.rename({"x": "longitude", "y": "latitude", "lon": "longitude", "lat": "latitude"})
regrid_cmip

In [ ]:
regrid_unep = functions_creche.xesmf_regrid(unep_pointzero1, lat_range=(min(lats), max(lats)), lon_range=(min(lons), max(lons)), resolution=1)

In [ ]:
spatial_plots.plot_spatial(regrid_cmip["mlotst_april_max"])

In [ ]:
spatial_plots.plot_spatial(regrid_unep["UNEP_GDCR"])

In [ ]:
lats = cmip["mlotst_january_mean"].latitude.values
lons = cmip["mlotst_january_mean"].longitude.values
resampled = functions_creche.xesmf_regrid(cmip, lat_range=(min(lats),max(lats)), lon_range=(min(lons),max(lons)), resolution=1)

In [ ]:
import xarray as xa

new_ds = resampled.copy()
new_ds = new_ds.rename({"x": "longitude", "y": "latitude"})
new_ds
# new_ds.coords["longitude"] = cmip["mlotst_january_mean"].lon.values[0,:]
# new_ds.coords["latitude"] = cmip["mlotst_january_mean"].lat.values[:,0]

# new_ds.reset_coords(["lon", "lat"], drop=True)
# new_ds.drop_dims(["x", "y"])

In [ ]:
# rename x and y to lon and lat
new_ds = new_ds.rename({"x": "longitude", "y": "latitude"})
new_ds

In [ ]:
from coralshift.plotting import spatial_plots
spatial_plots.plot_spatial(new_ds["mlotst_january_mean"])

In [ ]:
unep = get_data.ReturnRaster(
    dataset="unep",
    config_info=config_info).return_raster()
unep["UNEP_GDCR"].plot()

In [ ]:
gebco

In [ ]:
gebco = get_data.ReturnRaster(
    dataset="gebco",
    config_info=config_info).return_raster()
gebco["elevation"].plot()

In [ ]:
gebco["elevation"].values

In [ ]:
gebco

In [ ]:
from coralshift.plotting import spatial_plots
spatial_plots.plot_spatial(gebco["elevation"])

In [ ]:
unep

In [ ]:
gebco

In [ ]:
cmip

In [ ]:
c

In [ ]:
cmip.reset_coords()["mlotst_january_mean"].plot()

In [ ]:
reseted = cmip.reset_coords()

In [ ]:
ds = reseted.assign_coords(latitude=reseted.lat, longitude=reseted.lon)


In [ ]:
ds

In [ ]:
ds.latitude.values[:,0]

In [ ]:
ds["mlotst_january_mean"].values

In [ ]:
lat_vals = list(ds.latitude.values[:,0])
lon_vals = list(ds.longitude.values[0,:])

out = xa.Dataset(
    {'data': (['latitude', 'longitude'], [lat_vals], [lon_vals])},  # Example data variable
    coords={'latitude': lat_vals, 'longitude': lon_vals}
)

# ds = xr.Dataset(
#     {'data': (['latitude', 'longitude'], [[1, 2, 3, 4], [5, 6, 7, 8]])},  # Example data variable
#     coords={'latitude': latitude_values, 'longitude': longitude_values}
# )

In [ ]:

ds.longitude.values[0,:]


In [ ]:
import cartopy.crs as ccrs
ax = plt.axes(projection=ccrs.PlateCarree())

cmip["mlotst_january_mean"].plot.pcolormesh(
    ax=ax, transform=ccrs.PlateCarree(), x="lon", y="lat", add_colorbar=False
)

In [ ]:
import xarray as xa
# merged = xa.merge([gebco["elevation"], cmip["mlotst_january_mean"]])
merged = xa.merge([gebco["elevation"], unep["UNEP_GDCR"]])
merged["elevation"].plot()

In [ ]:
resampled = get_data.ProcessML(
    config_info=config_info,
    datasets=["gebco"]
).get_merged_datasets()
resampled["elevation"].plot()

In [ ]:
trains

In [ ]:
from coralshift import functions_creche
out = functions_creche.pd_to_array(trains[1])

out.plot()

In [ ]:
# cast dataframe to xarray dataset

trains[0]["mlotst_april_max"].to_xarray().plot()

In [ ]:
trains[0].to_xarray()

In [ ]:
df = ds.to_dataframe().dropna()

In [ ]:
from coralshift import functions_creche
functions_creche.adaptive_depth_mask(df, depth_mask_lims=[-50,0], pos_neg_ratio=0.5)

In [ ]:
def compute_value(x, proportion=0.1):
    if isinstance(x, float):
        return proportion * x
    elif isinstance(x, int):
        # Find the closest integer to proportion of the value (which isn't x)
        if x > 0:
            return round(x + proportion)
        elif x < 0:
            return round(x - proportion)
        else:
            return proportion  # Special case when x is 0
    else:
        raise ValueError("Input must be either float or integer")

def generate_gridsearch_parameter_grid(params_dict: dict, num_samples: int = 2) -> dict:
    grid_params = {}
    for key, value in params_dict.items():
        if key == "verbose":
            grid_params[key] = [value]
        elif isinstance(value, bool) or isinstance(value, str):
            grid_params[key] = [value]
        elif isinstance(value, int) or isinstance(value, float):
            if num_samples == 0:
                raise ValueError("num_samples cannot be zero")
            elif num_samples % 2 == 0:
                # For even num_samples, space the values around the original value
                step = compute_value(value)
                above_below_values = [value + 1 for value in values] + [value - 1 for value in values]
                # Ensure all remains as int if they started off that way
                if isinstance(value, int):
                    values = [int(val) for val in values]
                grid_params[key] = values
            else:
                # For odd num_samples, include the original value and have equal steps around it
                step = round(abs(value) / (num_samples // 2), 2)
                values = [
                    round(value - step * i, 2)
                    for i in range(num_samples // 2, -num_samples // 2 - 1, -1)
                ]
                # Ensure all remains as int if they started off that way
                if isinstance(value, int):
                    values = [int(val) for val in values]
                grid_params[key] = values
    return grid_params


params_dict = {"val": 2}

generate_gridsearch_parameter_grid(params_dict, num_samples=2)

In [ ]:
# config_info = file_ops.read_yaml("/maps/rt582/coralshift/runs/0-1d/xgb_reg/ID002_cmip6_unep_gebco_CONFIG.yaml")
config_info = file_ops.read_yaml("/maps/rt582/coralshift/config_test.yaml")
model = get_data.RunML(
    trains=trains,
    tests=tests,
    vals=vals,
    # regressor_classification_threshold=config_info["regressor_classification_threshold"],
    # n_samples=config_info["hyperparameter_search"]["n_samples"],
    # model_code=config_info["model_code"],
    # cv_folds=config_info["hyperparameter_search"]["cv_folds"],
    # n_iter=config_info["hyperparameter_search"]["n_iter"],
    # do_search=config_info["hyperparameter_search"]["do_search"],
    # do_save_model=config_info["do_save_model"],
    # do_train=config_info["do_train"],
    # param_search_type=config_info["hyperparameter_search"]["type"],
    config_info=config_info).run_model()

In [ ]:
config_info = file_ops.read_yaml("/maps/rt582/coralshift/config_test.yaml")
# check if grid and random search are in search_types
"grid" and "asdf" in config_info["hyperparameter_search"]["search_types"]

In [ ]:
model = file_ops.read_pkl("/maps/rt582/coralshift/runs/0-1d/xgb_reg/ID004_cmip6_unep_gebco_MODEL.pickle")
confusion_values = get_data.AnalyseResults(
    model=model,
    trains=trains,
    tests=tests,
    vals=vals,
    conf_mat=True,
    config_info=file_ops.read_yaml("/maps/rt582/coralshift/runs/0-1d/xgb_reg/ID002_cmip6_unep_gebco_CONFIG.yaml")
).plot_spatial_confusion_matrix(X=trains[0], y=trains[1])

In [ ]:
model = file_ops.read_pkl("/maps/rt582/coralshift/runs/0-1d/gb_reg/ID002_cmip6_unep_gebco_MODEL.pickle")

ar_object = get_data.AnalyseResults(
    model=model,
    trains=trains,
    tests=tests,
    vals=vals,
    conf_mat=True,
    config_info=file_ops.read_yaml("/maps/rt582/coralshift/runs/0-1d/gb_reg/ID002_cmip6_unep_gebco_CONFIG.yaml")
)


predictions = ar_object.make_predictions(trains[0])
ar_object.plot_regression(y=trains[1], predictions=predictions)

# confusion_values = get_data.AnalyseResults(
#     model=model,
#     trains=trains,
#     tests=tests,
#     vals=vals,
#     conf_mat=True,
#     config_info=file_ops.read_yaml("/maps/rt582/coralshift/runs/0-1d/gb_reg/ID002_cmip6_unep_gebco_CONFIG.yaml")
# ).plot_regression(y=trains[1], predictions=predictions)

In [ ]:
model = file_ops.read_pkl("/maps/rt582/coralshift/runs/0-1d/gb_reg/ID002_cmip6_unep_gebco_MODEL.pickle")
confusion_values = get_data.AnalyseResults(
    model=model,
    trains=trains,
    tests=tests,
    vals=vals,
    conf_mat=True,
    config_info=file_ops.read_yaml("/maps/rt582/coralshift/runs/0-1d/gb_reg/ID002_cmip6_unep_gebco_CONFIG.yaml")
).analyse_results()

In [ ]:
predictions = get_data.AnalyseResults(
    model=model,
    trains=trains,
    tests=tests,
    vals=vals,
    conf_mat=True,
    config_info=file_ops.read_yaml("/maps/rt582/coralshift/runs/0-1d/xgb_reg/ID000_cmip6_unep_gebco_CONFIG.yaml")
).plot_confusion_matrix(X=trains[0], y=trains[1])


In [ ]:
model_config_fps = [
    "/maps/rt582/coralshift/runs/0-1d/mlp_cf/ID000_cmip6_unep_gebco_CONFIG.yaml",
    "/maps/rt582/coralshift/runs/0-1d/mlp_reg/ID00_cmip6_unep_gebco_CONFIG.yaml"
]

get_data.run_config_files(model_config_fps)

In [ ]:
fig, ax = plt.subplots(dpi=300)

from coralshift.plotting import spatial_plots
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cmap = spatial_plots.get_cbar()

thresholded_pred = functions_creche.cont_to_class(
                    predictions, config_info["regressor_classification_threshold"]
                )
thresholded_act = functions_creche.cont_to_class(
                    trains[1], config_info["regressor_classification_threshold"]
                )

cm = confusion_matrix(thresholded_act, thresholded_pred, labels=[0, 1], normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=["absence", "presence"],
                             )

functions_creche.customize_plot_colors(fig,ax)
disp.plot(ax=ax, cmap=cmap, text_kw={"c":"k", "size": 16, "visible": True}, colorbar=False)

ax.set_xlabel('');
ax.set_ylabel('');
# print("Threshold: {}".format(threshold))

In [ ]:
from coralshift import functions_creche
plt.hist(functions_creche.cont_to_class(tests[1], 0.1))

In [ ]:
plt.hist(tests[1])

## Collate data
First check that data for current extent exists.

- Ground truths
    - UNEP-WCMC – look for raster at correct resolution. If not exists, generate and save
    - TODO: ACA – look for raster at
- Bathymetry
    - GEBCO – look for raster covering correct area. If not exists, look for .nc with correct (subsetting) extent. If this not present, throw error.
- Environmental variables
    - CMIP – look in env_vars subdirs for dir with encapsulating all of spatial_extent. If not exists, throw error. If exists, look for rf_all_vars_{res}_{extent}.nc file where extent fully encapsulates spatial_extent. If not exists, create from concatted_vars folder.

In [ ]:
# example loading in ground truth at required resolution
unep = get_data.ReturnRaster("unep", resolution=1).get_resampled_raster()

In [ ]:
gebco = get_data.ReturnRaster("gebco", lats=lats, lons=lons).get_resampled_raster()

In [ ]:
elevation = get_data.ReturnRaster("bathymetry", resolution=0.1, lats=lats, lons=lons).get_raw_raster()
elevation

In [ ]:
elevation = get_data.ReturnRaster("bathymetry", resolution=0.1, lats=[-30,0], lons=[130,170]).get_resampled_raster()
elevation

In [ ]:
# TODO: spatial buffering of data (at edges)
# TODO: mloess buffering of climate data
cmip = get_data.ReturnRaster("cmip", lats=[-40,0], lons=[130,170]).get_raw_raster()
cmip

In [ ]:
cmip = get_data.ReturnRaster("cmip", lats=[-70,0], lons=[130,170], spatial_buffer=2).return_raster()
cmip

In [ ]:
static_ds = get_data.ReturnRaster("cmip", lats=[-40,-20], lons=[150,170]).get_raw_raster()
static_ds

In [ ]:
trains, tests, vals = get_data.ProcessML().generate_ml_ready_data()

In [ ]:
train, test, val = get_data.RunML().threshold_data(trains, tests, vals)

In [ ]:
train[0].shape

In [ ]:
model_code = "xgbasdf"

"xgb" in model_code

In [ ]:
from dask.distributed import get_client, Client, LocalCluster
cluster = LocalCluster(n_workers=4)
client = Client(cluster)


get_data.RunML(trains=trains, tests=tests, vals=vals, model_code="logreg", param_search=True).run_model()

In [ ]:
static_ds.tos.isel(time=0).plot()

In [ ]:
s_train, s_test, s_val = get_data.ProcessML(model_type="timeseries", datasets=["unep", "cmip", "gebco", "gebco_slope"], lats=[-40,0], lons=[130,170]).generate_scaled_data()

In [ ]:
# generate an array of 1000 elements between 0 and 1 with a unimodal distribution close to 0.2
from sklearn.preprocessing import PowerTransformer, QuantileTransformer

test = np.random.triangular(0, 0.9, 1, 1000)
fitted = PowerTransformer().fit(test.reshape(-1, 1))
trans_test = fitted.transform(test.reshape(-1, 1))

import matplotlib.pyplot as plt

f, ax = plt.subplots(ncols=2)
ax[0].hist(test)
ax[1].hist(logged_test)

for a in [ax[0], ax[1]]:
    a.set_xlim(0, 1)

In [ ]:
plt.hist(np.random.normal(0.2, 0.05, 1000000))

In [ ]:
import xarray as xa
xa.open_dataset("/maps/rt582/coralshift/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/testing/thetao_mlotst_uo_vo_so_tos_rsdo_N0_S-40_W130_E170_levs_0-20_ll_195001-201412.nc")

In [ ]:
nc_fps= list(Path("/maps/rt582/coralshift/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/testing/concatted_vars_N0_S-50_W130_E170").glob("*.nc"))
[xa.open_dataset(fp) for fp in nc_fps[:]]

In [ ]:
import subprocess
import os

get_data.execute_subprocess_command(command, "/maps/rt582/coralshift/testlogs/cmip6_download_logs/SH-EC-Earth3P-HR_rsdo.txt", "/maps/rt582/coralshift/testlogs/cmip6_download_logs/SH-EC-Earth3P-HR_rsdo.txt")
# result = subprocess.run(
#             command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, check=True
#         )

# os.system(" ".join(command))

In [ ]:
result

In [ ]:
get_data.ReturnRaster("cmip6", env_vars=["rsdo"]
# , lats=[-41, 0], lons=[129,170]
).get_raw_raster()

In [ ]:
import xarray as xa
# xa.open_dataset("/maps/rt582/coralshift/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/testing/rsdo_mlotst_so_thetao_uo_vo_tos_N90.0_S-90.0_W-180.0_E180.0_levs_0-20_ll_195001-201412.nc").isel(time=0)["thetao"].plot()

xa.open_dataset("/maps/rt582/coralshift/data/env_vars/cmip6/EC-Earth3P-HR/r1i1p2f1/testing/concatted_vars_N0_S-50_W130_E170/vo_N0_S-50_W130_E170_sfl-20_ll_195001-201412.nc")